# 3ο Εργαστήριο Αναγνώριση Προτύπων

❐ Δημήτριος Κοκκίνης: 03118896

❐ Χριστίνα Ρεντίφη: 03118217

Αρχικά κάνουμε import τις απαραίτητες βιβλιοθήκες και εισάγουμε τα δεδομένα στο kaggle

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import copy

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader

import re

import joblib

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/patreco3-multitask-affective-music/data/'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## ΒΗΜΑ 1

### Α)

In [ ]:
spec1 = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train/100839.fused.full.npy')

spec2 = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/train/116543.fused.full.npy')

This is a fused spectrogram + chromagram. The shape is (mel + chroma frequencies, timesteps)

In [ ]:
print(spec1.shape)  # (frequencies x time steps)

print(spec2.shape)

### B)

To decompose into the mel spectrogram and chromagram you can run:

In [ ]:
mel1, chroma1 = spec1[:128], spec1[128:]

mel2, chroma2 = spec2[:128], spec2[128:]

### Γ)

In [ ]:
# Plot the spectrograms

fig1, ax1 = plt.subplots()
img1 = librosa.display.specshow(mel1, x_axis='time', y_axis='linear', ax=ax1)
ax1.set(title='Spectrogram 1')
fig1.colorbar(img1, ax=ax1, format="%+2.f dB")

fig2, ax2 = plt.subplots()
img2 = librosa.display.specshow(mel2, x_axis='time', y_axis='linear', ax=ax2)
ax2.set(title='Spectrogram 2')
fig2.colorbar(img2, ax=ax2, format="%+2.f dB")

Τα φασματογραφήματα που απεικονίζονται παραπάνω μας δείχνουν την φασματική πυκνότητα του ήχου. 
Βλέπουμε δύο διαστάσεις όπως φαίνεται και παραπάνω και η τρίτη διάσταση 
παρουσιάζεται με την μορφή χρώματος. Στον οριζόντιο άξονα έχουμε την διάσταση του χρόνου (αυξάνεται από 
αριστερά προς τα δεξιά). Ο κατακόρυφος άξονας αντιστοιχεί στην συχνότητα 
και μπορεί να θεωρηθεί ως το pitch (τόνος) με τις χαμηλότερες 
συχνότητες να ξεκινάνε από κάτω και να αυξάνονται καθώς ανεβαίνουμε 
κατακόρυφα. Το χρώμα που αντιπροσωπεύει την τρίτη διάσταση μας δείχνει το 
πλάτος (ή ενέργεια ή διαφορετικά ένταση ”loudness”) μιας δεδομένης 
συχνότητας σε μια δεδομένη χρονική στιγμή. Τα πιο σκούρα χρώματα 
αντιπροσωπεύουν χαμηλότερα πλάτη και τα πιο ανοιχτά πιο 
υψηλά πλάτη.

## BHMA 2

### A)

In [ ]:
print(mel1.shape, mel2.shape)
print()
print("Timesteps of spectogram 1 are: " + str(mel1.shape[1]))
print("Timesteps of spectogram 2 are: " + str(mel2.shape[1]))

Έχουμε πολλά χρονικά βήματα οπότε πρέπει κάπως να τα μειώσουμε ώστε να μπορέσουμε να εκπαιδέυσουμε ένα LSTM

### Β)

In [ ]:
spec1 = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train/100839.fused.full.npy')

spec2 = np.load('/kaggle/input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train/116543.fused.full.npy')

In [ ]:
print(spec1.shape)
print(spec2.shape)

In [ ]:
# mel spectrogram + chromagram, timesteps
mel1_beat, chroma1_beat = spec1[:128], spec1[128:]
mel2_beat, chroma2_beat = spec2[:128], spec2[128:]

In [ ]:
# Plot the spectrograms

fig1, ax1 = plt.subplots()
img1 = librosa.display.specshow(mel1_beat, x_axis='time', y_axis='linear', ax=ax1)
ax1.set(title='Spectrogram 1')
fig1.colorbar(img1, ax=ax1, format="%+2.f dB")

fig2, ax2 = plt.subplots()
img2 = librosa.display.specshow(mel2_beat, x_axis='time', y_axis='linear', ax=ax2)
ax2.set(title='Spectrogram 2')
fig2.colorbar(img2, ax=ax2, format="%+2.f dB")

Παρατηρούμε τώρα ότι τα γραφήματα δεν είναι τόσο smooth, δεν έχουν τόσο μεγάλη ακρίβεια καθώς έχουν μειωθεί οι διαστάσεις με αποτέλεσμα να φαίνονται πιο "pixelated". Ωστόσο η πληροφορία δεν χάνεται σημαντικά επομένως μπορούμε να εκπαιδεύσουμε το LSTM πολύ πιο αποδοτικά.

## ΒΗΜΑ 3

In [ ]:
# Chromograms, 12 musical notes
# timesteps --> 2nd dimension
print(chroma1.shape) 
print(chroma2.shape)

print(chroma1_beat.shape)
print(chroma2_beat.shape)

In [ ]:
fig1, ax1 = plt.subplots()
img1 = librosa.display.specshow(chroma1, x_axis='time', y_axis='linear', ax=ax1)
ax1.set(title='Chromagram 1')
fig1.colorbar(img1, ax=ax1, format="%+2.f dB")

fig2, ax2 = plt.subplots()
img2 = librosa.display.specshow(chroma2, x_axis='time', y_axis='linear', ax=ax2)
ax2.set(title='Chromagram 2')
fig2.colorbar(img2, ax=ax2, format="%+2.f dB")

In [ ]:
#Chromagrams after dimensionality reduction

fig1, ax1 = plt.subplots()
img1 = librosa.display.specshow(chroma1_beat, x_axis='time', y_axis='linear', ax=ax1)
ax1.set(title='Spectrogram 1')
fig1.colorbar(img1, ax=ax1, format="%+2.f dB")

fig2, ax2 = plt.subplots()
img2 = librosa.display.specshow(chroma2_beat, x_axis='time', y_axis='linear', ax=ax2)
ax2.set(title='Spectrogram 2')
fig2.colorbar(img2, ax=ax2, format="%+2.f dB")

## ΒΗΜΑ 4

## Creating a Pytorch Dataset

We provide helper code that creates a pytorch dataset using the competition data:

In [ ]:
# Combine similar classes and remove underrepresented classes
class_mapping = {
    'Rock': 'Rock',
    'Psych-Rock': 'Rock',
    'Indie-Rock': None,
    'Post-Rock': 'Rock',
    'Psych-Folk': 'Folk',
    'Folk': 'Folk',
    'Metal': 'Metal',
    'Punk': 'Metal',
    'Post-Punk': None,
    'Trip-Hop': 'Trip-Hop',
    'Pop': 'Pop',
    'Electronic': 'Electronic',
    'Hip-Hop': 'Hip-Hop',
    'Classical': 'Classical',
    'Blues': 'Blues',
    'Chiptune': 'Electronic',
    'Jazz': 'Jazz',
    'Soundtrack': None,
    'International': None,
    'Old-Time': None
}

def torch_train_val_split(
        dataset, batch_train, batch_eval,
        val_size=.2, shuffle=True, seed=None):
    # Creating data indices for training and validation splits:
    
    dataset_size = len(dataset) #length of dataset
    indices = list(range(dataset_size)) #indices for each element in dataset [0,1,2,..., length of dataset-1]
    val_split = int(np.floor(val_size * dataset_size)) #στρογγυλοποίηση του val_size*dataset_size
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    # χωρισμός των δεικτών που θα είναι για το train set και το validation set
    train_indices = indices[val_split:] 
    val_indices = indices[:val_split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset,
                              batch_size=batch_train,
                              sampler=train_sampler)
    val_loader = DataLoader(dataset,
                            batch_size=batch_eval,
                            sampler=val_sampler)
    return train_loader, val_loader



# Helper functions to read fused, mel, and chromagram
def read_fused_spectrogram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)
    return spectrogram.T


def read_mel_spectrogram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)[:128]
    return spectrogram.T

    
def read_chromagram(spectrogram_file):
    spectrogram = np.load(spectrogram_file)[128:]
    return spectrogram.T


class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])
        
        
class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[:self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1
        
        
# Pytorch Dataset Class for creating the dataset
class SpectrogramDataset(Dataset):
    def __init__(self, path, class_mapping=None, train=True, max_length=-1, read_spec_fn=read_fused_spectrogram):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.files, labels = self.get_files_labels(self.index, class_mapping)
        self.feats = [read_spec_fn(os.path.join(p, f)) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        self.label_transformer = LabelTransformer()
        if isinstance(labels, (list, tuple)):
            self.labels = np.array(self.label_transformer.fit_transform(labels)).astype('int64')

    def get_files_labels(self, txt, class_mapping):
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
        files, labels = [], []
        for l in lines:
            label = l[1]
            if class_mapping:
                label = class_mapping[l[1]]
            if not label:
                continue
            # Kaggle automatically unzips the npy.gz format so this hack is needed
            _id = l[0].split('.')[0]
            npy_file = '{}.fused.full.npy'.format(_id)
            files.append(npy_file)
            labels.append(label)
        return files, labels

    def __getitem__(self, item):
        # TODO: Inspect output and comment on how the output is formatted
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)
    
    
# Create a pytorch dataset for beat synced mel spectrograms
if __name__ == "__main__":
    ##################################################################################
    # load beat synced mel spectrograms
    ##################################################################################
    # Dataset
    beat_mel_specs = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=True,
                                         class_mapping=class_mapping, max_length=-1,
                                         read_spec_fn=read_mel_spectrogram)
    # Train and Test loaders
    train_loader_beat_mel, val_loader_beat_mel = torch_train_val_split(beat_mel_specs, 32, 32, val_size=.33)
    test_dataset_beat_mel = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=False,
                                                 class_mapping=class_mapping, max_length=-1,
                                                 read_spec_fn=read_mel_spectrogram)
    test_loader_beat_mel = DataLoader(test_dataset_beat_mel, batch_size=1)
    
    ##################################################################################
    # load beat synced chroma chromagrams
    ##################################################################################
    beat_chroma = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=True,
                                     class_mapping=class_mapping, max_length=-1,
                                     read_spec_fn=read_chromagram)
    train_loader_beat_chroma, val_loader_beat_chroma = torch_train_val_split(beat_chroma, 32, 32, val_size=.33)
    test_dataset_beat_chroma = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/', train=False,
                                                 class_mapping=class_mapping, max_length=-1,
                                                 read_spec_fn=read_chromagram)
    test_loader_beat_chroma = DataLoader(test_dataset_beat_chroma, batch_size=1)

    ##################################################################################
    # load fused speectrogram + chromagram for the full (non-beat-synced) data
    ##################################################################################
    specs_fused = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=True,
                                     class_mapping=class_mapping, max_length=-1,
                                     read_spec_fn=read_fused_spectrogram)
    train_loader, val_loader = torch_train_val_split(specs_fused, 32, 32, val_size=.33)
    test_dataset = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=False,
                                     class_mapping=class_mapping, max_length=-1,
                                     read_spec_fn=read_fused_spectrogram)
    test_loader = DataLoader(test_dataset, batch_size=1)
    
    ##################################################################################
    # load single synced mel spectrograms
    ##################################################################################
    # Dataset
    mel_specs = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=True,
                                        class_mapping=class_mapping, max_length=-1,
                                        read_spec_fn=read_mel_spectrogram)
    # Train and Test loaders
    train_loader_mel, val_loader_mel = torch_train_val_split(mel_specs, 32, 32, val_size=.33)
    test_dataset_mel = SpectrogramDataset('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms/', train=False,
                                            class_mapping=class_mapping, max_length=-1,
                                            read_spec_fn=read_mel_spectrogram)
    
    test_loader_mel = DataLoader(test_dataset_mel, batch_size=1)

Το όρισμα dataset μας επιτρέπει να ορίσουμε εμείς το μέγεθος της ακολουθίας που τελικά θα χρησιμοποιήθει. Με αυτό τον τρόπο μπορούμε να τρέξουμε το νευρωνικό δίκτυο με batches. Στην μελέτη μας, επιλέγουμε να το ορίσουμε ως το μέγιστο μήκος της ακολουθίας.

Όταν κάνουμε πειράματα (debugging) με τα μοντέλα, δεν πρέπει να έχουμε καμία τυχαιότητα στην επιλογή των δύο dataset (ισοδύναμα seed=None). Αυτό συμβαίνει, διότι θέλουμε να ελέγχουμε την επίδοση διάφορων υπερπαραμέτρων (batch size, size of layers, etc) και αν επιτρέπουμε τυχαιότητα στο dataset, δεν θα είναι δυνατό να ερμηνεύσουμε βελτίωση στο accuracy.

**Lets see an example sample from the dataset**

In [ ]:
datum = next(iter(train_loader_beat_mel))
print('Data shape')
print(datum[0].shape)  # shape of data
print('Labels')
print(datum[1])  # labels in batch
print('Lengths')
print(datum[2])  # length of each element in batch

**Lets see an example sample from the beat chroma dataset**

In [ ]:
datum = next(iter(train_loader_beat_chroma))
print('Data shape')
print(datum[0].shape)  # shape of data
print('Labels')
print(datum[1])  # labels in batch
print('Lengths')
print(datum[2])  # length of each element in batch

#### Create a pytorch dataset for the fused spectrogram + chromagram for the full (non-beat-synced) data

In [ ]:
datum = next(iter(train_loader))
print('Data shape')
print(datum[0].shape)  # shape of data
print('Labels')
print(datum[1])  # labels in batch
print('Lengths')
print(datum[2])  # length of each element in batch

# Ιστογράμματα

## Before mapping

In [ ]:
y_train = []
y_test = []

# Train labels
file1 = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train_labels.txt', 'r') 
Lines = file1.readlines()[1:] 
for line in Lines: 
    label = line.split()[1]
    y_train.append(label)
    
# Test labels
file1 = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/test_labels.txt', 'r') 
Lines = file1.readlines()[1:] 
for line in Lines: 
    label = line.split()[1]
    y_test.append(label)

# before mapping
bins = np.arange(len(np.unique(y_train))+1) - 0.5
# train
plt.figure(figsize=(24,8))
plt.title('y_train', fontsize=16)
n, bins, patches = plt.hist(y_train, bins, facecolor='blue', rwidth=0.25)
plt.show()

In [ ]:
# before mapping
bins = np.arange(len(np.unique(y_test))+1) - 0.5
# test
plt.figure(figsize=(24,8))
plt.title('y_test', fontsize=16)
n, bins, patches = plt.hist(y_test, bins, facecolor='blue', rwidth=0.25)
plt.show()

## After mapping

In [ ]:
y_train = []
y_test = []

# Train labels
file1 = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/train_labels.txt', 'r') 
Lines = file1.readlines()[1:] 
for line in Lines: 
    label = line.split()[1]
    if class_mapping[label] is not None:
        y_train.append(class_mapping[label])
    
# Test labels
file1 = open('../input/patreco3-multitask-affective-music/data/fma_genre_spectrograms_beat/test_labels.txt', 'r') 
Lines = file1.readlines()[1:] 
for line in Lines: 
    label = line.split()[1]
    if class_mapping[label] is not None:
        y_test.append(class_mapping[label])
        
# after mapping
bins = np.arange(len(np.unique(y_train))+1) - 0.5
# train
plt.figure(figsize=(24,8))
plt.title('y_train', fontsize=16)
n, bins, patches = plt.hist(y_train, bins, facecolor='red', rwidth=0.75)
plt.show()

In [ ]:
# after mapping
bins = np.arange(len(np.unique(y_test))+1) - 0.5
# test
plt.figure(figsize=(24,8))
plt.title('y_test', fontsize=16)
n, bins, patches = plt.hist(y_test, bins, facecolor='red', rwidth=0.75)
plt.show()